In [27]:
import pandas_datareader as pdr
import numpy as np
import pandas as pd
import scipy.optimize 
from datetime import datetime,time,timedelta
from dateutil import relativedelta
from strategy2 import strategy2


In [28]:
def optimize(stock_return, weights,tickers,start,end,calc_window,rf_rate,short=False,
             price_type='Close',clean='True'):
          
        '''
        weights; *type = list, initial guess weights
        
        start; input as str, data start date format 'YYYY-MM-DD' 
        end; input as str, data end date format 'YYYY-MM-DD'
        
        calc_window; input as str, calculation window for returns, 
        
        rf_rate; desired risk-free rate to benchmark against
        
        price_type; *input as str, default = 'Close', options = 'Adj Close', 
        'Close', 'High','Low','Open', 'Volume'
        
        short = bool, default = 'False', if true weight bounds -1 to 1
        
        clean; bool, default = 'False', if true, the function will drop NA rows
        '''
        start = datetime.strptime(start,'%Y-%m-%d')
        end = datetime.strptime(end,'%Y-%m-%d')
        
        prices = pdr.get_data_yahoo(tickers, start, end)[price_type]
        
        op = prices.pct_change(periods=int(calc_window))
        op.iloc[-1,:]=stock_return
        if clean == False:
            op
            
        else:
            op = op.dropna()
        
        vcv_matrix = np.cov(op.T)
        
        avg_return = op.mean()
        
        def sharpe(weights,vcv_matrix,avg_return):        
            
            var = np.dot(np.dot(weights,vcv_matrix),weights)
            
            ret =  np.dot(weights,avg_return)
            
            s_ratio = -(ret-rf_rate)/np.sqrt(var)
            
            return s_ratio
           
        
        sharpe_cons = ({'type':'eq','fun': lambda x: sum(x)-1})
        
        
        if short == False:
            sharpe_bnds = ((0,1),)*len(tickers)
        
        if short == True:
            sharpe_bnds = ((-1,1),)*len(tickers)
        
        optimal = scipy.optimize.minimize(sharpe,weights,bounds = sharpe_bnds,
                                 args = (vcv_matrix,avg_return),
                                 constraints = sharpe_cons)
        
        return optimal

In [29]:
def stock_list_return(date):
    factors = pd.read_csv("../../Factors/merge_Factors.csv")
    factor_returns_pred = pd.read_csv("../../Predict/factors_returns_pred.csv")
    factor_returns_before = pd.read_csv("../../Analysis/factors_returns_multi.csv")
    factor_returns_before = factor_returns_before.loc[factor_returns_before["Date"]<"2022-08-01"]
    factor_returns = factor_returns_before.append(factor_returns_pred)
    
    selected_stocks = strategy2(date)
    factors = factors.loc[(factors["Date"]==date) & (factors["Index"].isin(selected_stocks))]
    factor_returns = factor_returns.loc[factor_returns["Date"]==date]
    factor_returns = factor_returns.iloc[:,1:]
    stock_list = factors["Index"].tolist()
    factors = factors.iloc[:, 3:]
    factors = np.array(factors)
    factor_returns = np.array(factor_returns)

    stock_return = np.dot(factors, factor_returns.T).reshape(-1)
    stock_list = np.array(stock_list)
    return stock_return, stock_list

In [30]:
def get_weight(date):  

    # factors = pd.read_csv("../../Factors/merge_Factors.csv")
    # factor_returns = pd.read_csv("../../Predict/factors_returns_pred.csv")

    # selected_stocks = strategy2(date)
    # factors = factors.loc[(factors["Date"]==date) & (factors["Index"].isin(selected_stocks))]
    # factor_returns = factor_returns.loc[factor_returns["Date"]==date]
    # factor_returns = factor_returns.iloc[:,1:]
    # stock_list = factors["Index"].tolist()
    # factors = factors.iloc[:, 3:]
    # factors = np.array(factors)
    # factor_returns = np.array(factor_returns)

    # stock_return = np.dot(factors, factor_returns.T).reshape(-1)
    # stock_list = np.array(stock_list)
    stock_return, stock_list = stock_list_return(date)

    weights=[0.1]*10
    tickers=stock_list
    #end为要预测仓位的那天
    end=date
    #start为end向前一个月
    start=(pd.to_datetime(date)-relativedelta.relativedelta(months=1)).strftime("%Y-%m-%d")
    #不变，始终为1
    calc_window='1'


    # 确定无风险收益率
    rf=pd.read_csv('daily-treasury-rates.csv',index_col=0)

    #rf=rf.iloc[:,5]

    rf.reset_index(inplace=True)

    rf['Date']=[datetime.strptime(i,"%m/%d/%Y") for i in list(rf['Date'].values)]

    start_time=datetime.strptime(start,"%Y-%m-%d")
    end_time=datetime.strptime(end,"%Y-%m-%d")

    rf=rf.set_index("Date")


    for i in range(len(rf.index)):
        if list(rf.index)[i]-end_time<=timedelta(0):
            rfend_time=list(rf.index)[i]
            break
    for j in range(len(rf.index)):
        if list(rf.index)[j]-start_time<=timedelta(0):
            rfstart_time=list(rf.index)[j]
            break

    rf_rate=rf.loc[rfend_time:rfstart_time,'1 Yr'].mean()*0.01/360

    w=optimize(stock_return, weights,tickers,start,end,calc_window,rf_rate,short=True,price_type='Adj Close',clean='True')
    weight=list(w.x)
    return weight

In [31]:
get_weight("2022-07-25")

[0.16907433158779533,
 0.09081989428258834,
 0.5819192176566022,
 -0.5385230821133727,
 0.05425643538085306,
 0.9999999999999999,
 -0.37366654904476787,
 0.04269054991466533,
 0.09674492767754023,
 -0.12331572534190363]

In [46]:
def get_daily_return(date, stock_list):
    factors = pd.read_csv("../../Factors/merge_Factors.csv")

    selected_stocks = stock_list
    factors = factors.loc[(factors["Date"]==date) & (factors["Index"].isin(selected_stocks))]
    factor_returns = factor_returns.loc[factor_returns["Date"]==date]
    factor_returns = factor_returns.iloc[:,1:]
    stock_list = factors["Index"].tolist()
    factors = factors.iloc[:, 3:]
    factors = np.array(factors)
    factor_returns = np.array(factor_returns)

    stock_return = np.dot(factors, factor_returns.T).reshape(-1)

    # stock_return, stock_list = stock_list_return(date)
    factors = factors.iloc[:, :3]
    factors = factors.loc[factors["Date"]==date]
    factors = factors[factors["Index"].isin(stock_list)]
    true_return = np.array(factors["Return"])

    weight = get_weight(date)
    weight = np.array(weight)
    daily_return = sum(stock_return * weight)
    return daily_return

In [47]:
get_daily_return("2022-07-25")

0.011045579531432105

In [66]:
def get_monthly_return(date):
    factors = pd.read_csv("../../Factors/merge_Factors.csv")
    stock_return, stock_list = stock_list_return(date)

    date = pd.to_datetime(date)
    next_month_date = pd.to_datetime(date)+relativedelta.relativedelta(months=1)
    dates = factors["Date"].unique()
    dates = pd.to_datetime(dates)
    dates = dates[dates>=date]
    dates = dates[dates<next_month_date]
    dates = dates.strftime("%Y-%m-%d").tolist()

    monthly_return = []
    for daily in dates:
        daily_return = get_daily_return(daily, stock_list)
        monthly_return.append(daily_return)

    
    return monthly_return

In [67]:
get_monthly_return("2022-07-01")

KeyboardInterrupt: 

In [55]:
def annal_vala(r):
    return (r.var()**0.5)*(252**0.5)

Timestamp('2022-11-25 00:00:00')

In [ ]:
def month_to_year(t,end_net):
    k=12/t
    r=end_net-1
    year_r=(1+r)**k-1
    return year_r